In [18]:
import pandas as pd
import spacy
from tqdm import tqdm
from typing import List
from functools import lru_cache
from collections import Counter
import re
from spacy import displacy


tqdm.pandas()

### Загрузка обученных моделей

In [19]:
nlp_cat = spacy.load('../train/sym/sym_model/model-best')

In [20]:
nlp_a = spacy.load('../train/subsym/subsym_a/subsym_model/model-best/')

In [21]:
nlp_b = spacy.load('../train/subsym/subsym_b/subsym_model/model-best/')

In [22]:
nlp_c = spacy.load('../train/subsym/subsym_c/subsym_model/model-best/')

In [23]:
nlp_kw = spacy.load('../../spacy/train/ner_model/ner_model/model-best/')

### Загрузка данных

In [3]:
df = pd.read_excel('../../raw_data/CRA_test_422.xlsx')

In [4]:
def drop_tail(text):
    if 'агентство НКР' in text:
        return text.split('Регуляторное раскрытие')[0]
    elif 'Национальное Рейтинговое Агентство' in text:
        text = text.split('(далее – НРА, Агентство)')[1]
        return text.split('ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ')[0]
    elif 'АКРА' in text:
        return text.split('Регуляторное раскрытие')[0]
    elif 'Эксперт РА' in text:
        return text.split('Контакты для СМИ')[0]
    return text

In [5]:
df['clear_text'] = df['pr_txt'].progress_apply(drop_tail)

100%|██████████| 422/422 [00:00<00:00, 30992.76it/s]


In [6]:
df.sample(3)

,Id,pr_txt,Категория,Уровень рейтинга,clear_text
67,1279,«Эксперт РА» присвоил кредитный рейтинг компан...,NaN,NaN,«Эксперт РА» присвоил кредитный рейтинг компан...
356,1566,"\n\nЭкспертное агентство ""Финансовый анализ"" п...",NaN,NaN,"\n\nЭкспертное агентство ""Финансовый анализ"" п..."
12,1224,«Эксперт РА» присвоил кредитный рейтинг компан...,NaN,NaN,«Эксперт РА» присвоил кредитный рейтинг компан...


In [7]:
nlp = spacy.load('ru_core_news_lg')

In [8]:
@lru_cache(100000)
def clear(text):
    ner_list = ['ORG', 'LOC']
    result = []
    doc = nlp(text)
    for token in doc:
        if token.ent_type_ not in ner_list:
            result.append(token.text.lower())
    text =  ' '.join(result)
    text = text.replace('ё', 'е')
    text = text.replace('Ё', 'Е')
    text = re.sub(r'[^а-яА-Я ]', '', text)
    text = text.replace('\n', ' ')
    text = text.replace('.', '. ')
    text = text.replace(',', ', ')
    text = re.sub('\s+', ' ', text)
    return text.strip()
   

In [9]:
df['clear_text'] = df['clear_text'].progress_apply(clear)


100%|██████████| 422/422 [00:55<00:00,  7.59it/s]


In [10]:
df.sample(3)

,Id,pr_txt,Категория,Уровень рейтинга,clear_text
16,1228,АКРА повысило кредитный рейтинг ООО «РКС-Холди...,NaN,NaN,повысило кредитный рейтинг до прогноз стабильн...
298,1508,23 июля 2021 г.\n\nВедущий рейтинговый аналит...,NaN,NaN,июля г ведущий рейтинговый аналитик иванова ма...
254,1458,«Эксперт РА» снижает рейтинг кредитоспособнос...,NaN,NaN,снижает рейтинг кредитоспособности до уровня и...


In [11]:
@lru_cache(100000)
def lemmatize(text):
    result = []
    doc = nlp(text)
    for token in doc:
        result.append(token.lemma_)
    return ' '.join(result)

In [12]:
df['clear_text'] = df['clear_text'].progress_apply(lemmatize)


100%|██████████| 422/422 [00:44<00:00,  9.49it/s]


In [13]:
nlp.Defaults.stop_words.update({'год', 'уровень', 'агентство', 'руб', 'млрд', 'компания', 'рейтинг'})

In [14]:
@lru_cache(100000)
def tokenize_it(text: str) -> List:
    result = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop != True and token.is_punct != True:
            result.append(token.text)
    return result

In [15]:
df['tokenized'] = df['clear_text'].progress_apply(tokenize_it)

100%|██████████| 422/422 [00:38<00:00, 11.01it/s]


In [16]:
df['tokenized_str'] = df['tokenized'].apply(lambda x: ' '.join(x))

### Предсказание классов

In [24]:
def get_prection_cat(text):
    doc = nlp_cat(text)
    scores = doc.cats
    if max(scores, key=scores.get) == '1':
        doc = nlp_a(text)
        scores = doc.cats
        return max(scores, key=scores.get)
    elif max(scores, key=scores.get) == '2':
        doc = nlp_b(text)
        scores = doc.cats
        return max(scores, key=scores.get)
    elif max(scores, key=scores.get) == '3':
        doc = nlp_c(text)
        scores = doc.cats
        return max(scores, key=scores.get)    

In [25]:
df['Уровень рейтинга'] = df['tokenized_str'].apply(get_prection_cat)

In [26]:
df['Категория'] = df['Уровень рейтинга'].str.replace('+', '').str.replace('-', '')

C:\Users\wilte\AppData\Local\Temp\ipykernel_1868\3567747884.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Категория'] = df['Уровень рейтинга'].str.replace('+', '').str.replace('-', '')


In [27]:
submission = df[['Id', 'Категория', 'Уровень рейтинга']]

In [28]:
submission.to_csv('../../submission/submission.csv', sep=';')

### Выделение ключевых конструкций

In [35]:
def render_kw(text):
    OPTIONS  = {'colors': {'KEYWORDS': '#00FA9A'}}
    doc = nlp_kw(text)
    displacy.render(doc, style="ent", jupyter=True, options=OPTIONS)

In [43]:
render_kw(df['pr_txt'][15])